In [2]:
import json
import requests
from api_keys import gkey
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
from citipy import citipy
from scipy.stats import linregress
import csv
import gmaps
import os

In [3]:
#find the humidity's max number
weather_pd = pd.read_csv("../WeatherPy/weather_500_list.csv")
weather_pd['Humidity'].max()

100

In [4]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)

# Store 'Lat' and 'Lng' into  locations 
locations = weather_pd[["Latitude", "Longtitude"]].astype(float)

# Convert humidity number into float


Humidity_float = weather_pd["Humidity"].astype(float)  

fig = gmaps.figure()

#build the heatmap layer by using the locations and humidity_float
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity_float, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
weather_pd.head()

,Country,City,Longtitude,Latitude,Temperature,Cloudiness,Humidity,Wind Speed
0,NaN,Lüderitz,15.1594,-26.6481,295.31,54,50,3.62
1,CL,Punta Arenas,-70.9167,-53.1500,276.21,20,81,8.75
2,JP,Tanabe,135.3667,33.7333,297.00,75,80,0.45
3,TO,Vaini,-175.2000,-21.2000,296.24,75,94,3.09
4,RU,Provideniya,-173.3000,64.3833,275.30,100,98,3.58


In [6]:
# my condition: temperature between 270-290/ wind speed less than 3/cloudiness less than 20
greatest_condition_weather = weather_pd.loc[((weather_pd["Temperature"] > 270)&(weather_pd["Temperature"] < 290)&(weather_pd["Wind Speed"] < 3)&(weather_pd["Cloudiness"] <20))]
final_df = greatest_condition_weather.reset_index()
final_df = final_df.drop(columns = ['index'])
final_df

,Country,City,Longtitude,Latitude,Temperature,Cloudiness,Humidity,Wind Speed
0,BR,São Marcos,-51.0681,-28.9711,283.23,0,88,1.42
1,BR,Canguçu,-52.6756,-31.3950,282.79,0,82,1.49
2,US,Saint Paul Harbor,-152.4072,57.7900,278.88,1,87,2.06
3,GL,Qaanaaq,-69.3632,77.4840,273.61,16,79,1.59
4,RU,Pevek,170.3133,69.7008,273.71,12,84,2.13
5,PE,Pisco,-76.2167,-13.7000,286.18,7,88,2.57
6,US,Hamilton,-84.5333,39.1834,289.24,1,92,0.00
7,AR,Santiago del Estero,-64.2615,-27.7951,284.75,0,67,1.50
8,US,Jamestown,-79.2353,42.0970,287.25,1,92,0.45
9,BR,Santa Maria,-53.8069,-29.6842,284.76,0,82,2.06


In [7]:

for index,row in final_df.iterrows():
    lat = row['Latitude']
    lng = row['Longtitude']
# set up a parameters dictionary
    params = {"location": f"{lat},{lng}",
              "radius": 5000,
              "type": "lodging",
              "key": gkey
             }

# base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
    response = requests.get(base_url, params=params)
    places_data = response.json()
    try:
        hotel_name = places_data['results'][0]['name']
        final_df.loc[index, "Hotel"] = hotel_name
    except (KeyError, IndexError):
        print(f"there is no hotel whthin 5000m around {row['City']}")
        final_df.loc[index, "Hotel"] = " "
hotel_df = final_df.dropna(how = 'any')
hotel_df.head()

there is no hotel whthin 5000m around Palana


,Country,City,Longtitude,Latitude,Temperature,Cloudiness,Humidity,Wind Speed,Hotel
0,BR,São Marcos,-51.0681,-28.9711,283.23,0,88,1.42,Hotel São Marcos
1,BR,Canguçu,-52.6756,-31.3950,282.79,0,82,1.49,Casa Paulista
2,US,Saint Paul Harbor,-152.4072,57.7900,278.88,1,87,2.06,Best Western Kodiak Inn And Convention Center
3,GL,Qaanaaq,-69.3632,77.4840,273.61,16,79,1.59,Qaanaaq Hotel
4,RU,Pevek,170.3133,69.7008,273.71,12,84,2.13,Hotel ABC


In [8]:
#try to put a template 
info_box_template = """
<dl>
<dt>Hotel</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

#convert new locations into float and restore
locations_new = hotel_df[["Latitude", "Longtitude"]].astype(float)

#generate the marker layer
markers = gmaps.marker_layer(locations_new, info_box_content = hotel_info)


fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))